In [290]:
from pyspark.sql import SparkSession

In [291]:
from pyspark import SparkContext,SQLContext

In [398]:
from pyspark.sql.functions import countDistinct,max,stddev, col,sum, desc,count,mean

In [293]:
spark = SparkSession.builder.appName('tpm').getOrCreate()

In [294]:
rdd = spark.sparkContext.textFile('work/all_stocks_5yr.csv')
rdd

work/all_stocks_5yr.csv MapPartitionsRDD[793] at textFile at NativeMethodAccessorImpl.java:0

In [295]:
df = spark.read.option('header',True).csv('work/all_stocks_5yr.csv',sep=',')

In [296]:
df.show()

+----------+-----+-----+-----+-----+--------+----+
|      date| open| high|  low|close|  volume|Name|
+----------+-----+-----+-----+-----+--------+----+
|2013-02-08|15.07|15.12|14.63|14.75| 8407500| AAL|
|2013-02-11|14.89|15.01|14.26|14.46| 8882000| AAL|
|2013-02-12|14.45|14.51| 14.1|14.27| 8126000| AAL|
|2013-02-13| 14.3|14.94|14.25|14.66|10259500| AAL|
|2013-02-14|14.94|14.96|13.16|13.99|31879900| AAL|
|2013-02-15|13.93|14.61|13.93| 14.5|15628000| AAL|
|2013-02-19|14.33|14.56|14.08|14.26|11354400| AAL|
|2013-02-20|14.17|14.26|13.15|13.33|14725200| AAL|
|2013-02-21|13.62|13.95| 12.9|13.37|11922100| AAL|
|2013-02-22|13.57| 13.6|13.21|13.57| 6071400| AAL|
|2013-02-25| 13.6|13.76| 13.0|13.02| 7186400| AAL|
|2013-02-26|13.14|13.42| 12.7|13.26| 9419000| AAL|
|2013-02-27|13.28|13.62|13.18|13.41| 7390500| AAL|
|2013-02-28|13.49|13.63|13.39|13.43| 6143600| AAL|
|2013-03-01|13.37|13.95|13.32|13.61| 7376800| AAL|
|2013-03-04| 13.5|14.07|13.47| 13.9| 8174800| AAL|
|2013-03-05|14.01|14.05|13.71|1

In [297]:
rdd.count()

619041

In [298]:
df.count()

619040

In [299]:
df.summary().show()

+-------+----------+----------------+-----------------+-----------------+-----------------+-----------------+------+
|summary|      date|            open|             high|              low|            close|           volume|  Name|
+-------+----------+----------------+-----------------+-----------------+-----------------+-----------------+------+
|  count|    619040|          619029|           619032|           619032|           619040|           619040|619040|
|   mean|      null|83.0233343145481|83.77831069347307|82.25609641375267|83.04376276476548|4321823.395568945|  null|
| stddev|      null|97.3787690433234|98.20751890446383|96.50742105809076|97.38974800165785| 8693609.51196759|  null|
|    min|2013-02-08|            1.62|             1.69|              1.5|             1.59|                0|     A|
|    25%|      null|           40.22|            40.62|            39.83|            40.25|        1070186.0|  null|
|    50%|      null|           62.59|            63.15|         

In [300]:
df.filter(df.Name == 'AAPL').count()

1259

In [301]:
df2 = df.select(countDistinct('Name'))
df2.show()

+--------------------+
|count(DISTINCT Name)|
+--------------------+
|                 505|
+--------------------+



In [302]:
precoFechamentoMaior = df.filter(df.close > df.open).count()
precoFechamentoMaior

318940

In [303]:
318940/619040

0.5152171103644353

In [304]:
df.filter(df.Name == 'AAPL').show()

+----------+-------+-------+-------+-------+---------+----+
|      date|   open|   high|    low|  close|   volume|Name|
+----------+-------+-------+-------+-------+---------+----+
|2013-02-08|67.7142|68.4014|66.8928|67.8542|158168416|AAPL|
|2013-02-11|68.0714|69.2771|67.6071|68.5614|129029425|AAPL|
|2013-02-12|68.5014|68.9114|66.8205|66.8428|151829363|AAPL|
|2013-02-13|66.7442|67.6628|66.1742|66.7156|118721995|AAPL|
|2013-02-14|66.3599|67.3771|66.2885|66.6556| 88809154|AAPL|
|2013-02-15|66.9785|67.1656|65.7028|65.7371| 97924631|AAPL|
|2013-02-19|65.8714|66.1042|64.8356|65.7128|108854046|AAPL|
|2013-02-20|65.3842|65.3842|64.1142|64.1214|118891367|AAPL|
|2013-02-21|63.7142|64.1671|63.2599|63.7228|111596821|AAPL|
|2013-02-22|64.1785|64.5142|63.7999|64.4014| 82583823|AAPL|
|2013-02-25|64.8356|65.0171|63.2242|63.2571| 92899597|AAPL|
|2013-02-26|63.4028|64.5056|62.5228|64.1385|125096657|AAPL|
|2013-02-27|64.0614|64.6342|62.9499|63.5099|146674682|AAPL|
|2013-02-28|63.4357|63.9814|63.0571|63.0

In [305]:
df.filter(df.Name == 'AAPL').select(max('close')).show()

+----------+
|max(close)|
+----------+
|     99.99|
+----------+



In [306]:
df.dtypes

[('date', 'string'),
 ('open', 'string'),
 ('high', 'string'),
 ('low', 'string'),
 ('close', 'string'),
 ('volume', 'string'),
 ('Name', 'string')]

In [307]:
df_mod = df.withColumn('volume',df.volume.cast('double'))

In [308]:
df_mod.dtypes

[('date', 'string'),
 ('open', 'string'),
 ('high', 'string'),
 ('low', 'string'),
 ('close', 'string'),
 ('volume', 'double'),
 ('Name', 'string')]

In [309]:
maxVol = df_mod.select(max('volume')).show()
maxVol

+-----------+
|max(volume)|
+-----------+
|6.1823763E8|
+-----------+



Qual ação tem a maior volatilidade? Uma forma é medir o desvio-padrão do preço de fechamento
de cada ação e considerar a ação de maior desvio-padrão.

In [310]:
df_mod.groupBy('name').agg(stddev('close').alias('desvioPadrao')).sort(col('desvioPadrao').desc()).show()

+-----+------------------+
| name|      desvioPadrao|
+-----+------------------+
| PCLN|  320.533473018748|
| AMZN|  282.500395033522|
|GOOGL|187.57389185479704|
| GOOG|166.42052902768182|
|  MTD|131.39721989584996|
|  AZO|130.74959678201296|
|  CMG|130.28062354899413|
| EQIX| 95.21280494966273|
| REGN| 94.06934511650653|
| CHTR| 83.63699908227738|
| ISRG| 72.97599882123069|
| AVGO| 71.41622240006728|
| ULTA| 67.15184610039526|
|  NOC| 66.39592749504422|
|  SHW| 64.52058470708296|
|  LMT| 62.41202936228181|
|  BLK| 62.33688426948667|
| NVDA| 59.62464184686514|
| ORLY| 59.09508035311545|
|  AGN|59.026349830314544|
+-----+------------------+
only showing top 20 rows



Qual o dia com maior volume total de negociação da bolsa?

In [311]:
df3=df_mod.groupBy(col('date')).sum('volume')
df3.sort("sum(volume)", ascending=False).collect()

[Row(date='2015-08-24', sum(volume)=4607945196.0),
 Row(date='2016-06-24', sum(volume)=4367393052.0),
 Row(date='2015-12-18', sum(volume)=4124454411.0),
 Row(date='2016-01-20', sum(volume)=4087629753.0),
 Row(date='2018-02-06', sum(volume)=4072080890.0),
 Row(date='2016-11-10', sum(volume)=4060601612.0),
 Row(date='2014-10-15', sum(volume)=3993171524.0),
 Row(date='2013-06-21', sum(volume)=3983923288.0),
 Row(date='2015-09-18', sum(volume)=3962050449.0),
 Row(date='2016-11-09', sum(volume)=3915089371.0),
 Row(date='2016-01-15', sum(volume)=3787884056.0),
 Row(date='2017-12-15', sum(volume)=3786992731.0),
 Row(date='2016-02-08', sum(volume)=3759709109.0),
 Row(date='2016-03-18', sum(volume)=3755746256.0),
 Row(date='2016-01-29', sum(volume)=3740170664.0),
 Row(date='2014-03-21', sum(volume)=3611991561.0),
 Row(date='2018-02-05', sum(volume)=3598437288.0),
 Row(date='2013-03-15', sum(volume)=3569550899.0),
 Row(date='2016-12-16', sum(volume)=3564871264.0),
 Row(date='2016-02-11', sum(vol

Qual o maior valor das ações da Apple (AAPL) na história?

In [312]:
df_aple.columns

['date', 'open', 'high', 'low', 'close', 'volume', 'Name']

In [313]:
df_aple = df_aple.withColumn('high',df_aple.high.cast('double'))
df_aple = df_aple.withColumn('close',df_aple.high.cast('double'))
df_aple = df_aple.withColumn('low',df_aple.high.cast('double'))
df_aple = df_aple.withColumn('open',df_aple.high.cast('double'))

In [314]:
df_aple.dtypes

[('date', 'string'),
 ('open', 'double'),
 ('high', 'double'),
 ('low', 'double'),
 ('close', 'double'),
 ('volume', 'double'),
 ('Name', 'string')]

In [315]:
df_aple.groupBy('date').max('close').sort(desc('max(close)')).show()

+----------+----------+
|      date|max(close)|
+----------+----------+
|2018-01-18|     180.1|
|2018-01-19|    179.58|
|2018-01-23|    179.44|
|2018-01-16|    179.39|
|2018-01-17|    179.25|
|2018-01-22|    177.78|
|2018-01-12|    177.36|
|2018-01-24|     177.3|
|2017-12-18|     177.2|
|2017-11-08|    176.24|
|2017-11-09|   176.095|
|2017-12-21|    176.02|
|2018-01-08|    175.61|
|2017-11-24|     175.5|
|2018-01-11|  175.4886|
|2017-12-22|   175.424|
|2017-12-20|    175.42|
|2017-12-19|    175.39|
|2017-11-10|    175.38|
|2018-01-05|    175.37|
+----------+----------+
only showing top 20 rows



Qual a ação mais negociada da bolsa, em volume de transações?

In [316]:
df_mod = df_mod.withColumn('high',df_mod.high.cast('double'))
df_mod = df_mod.withColumn('close',df_mod.high.cast('double'))
df_mod = df_mod.withColumn('low',df_mod.high.cast('double'))
df_mod = df_mod.withColumn('open',df_mod.high.cast('double'))


In [317]:
df_mod.dtypes

[('date', 'string'),
 ('open', 'double'),
 ('high', 'double'),
 ('low', 'double'),
 ('close', 'double'),
 ('volume', 'double'),
 ('Name', 'string')]

Qual a ação mais negociada da bolsa, em volume de transações?

In [318]:
df_max_acaoVol = df_mod.groupBy(['Name']).agg(count('*'))
df_max_acaoVol.show()

+----+--------+
|Name|count(1)|
+----+--------+
|ALXN|    1259|
| AIV|    1259|
| AVY|    1259|
|BF.B|    1259|
| AXP|    1259|
|  CI|    1259|
| BLK|    1259|
| DGX|    1259|
|AAPL|    1259|
| ALL|    1259|
| CLX|    1259|
|CSCO|    1259|
| BMY|    1257|
| CMG|    1259|
| ADI|    1259|
| AGN|    1259|
|CHRW|    1259|
|AMGN|    1259|
| CCL|    1259|
|ANTM|    1259|
+----+--------+
only showing top 20 rows



In [322]:
df_max_acaoVol.summary().show()

+-------+----+------------------+
|summary|Name|          count(1)|
+-------+----+------------------+
|  count| 505|               505|
|   mean|null|1225.8217821782177|
| stddev|null|160.53327524567376|
|    min|   A|                44|
|    25%|null|              1259|
|    50%|null|              1259|
|    75%|null|              1259|
|    max| ZTS|              1259|
+-------+----+------------------+



In [323]:
df_max_acaoVol.dtypes

[('Name', 'string'), ('count(1)', 'bigint')]

In [333]:
df_mod.groupBy('Name').agg(sum('volume').alias('Volume_totalizado')).sort(desc('Volume_totalizado')).show()

+-----+-----------------+
| Name|Volume_totalizado|
+-----+-----------------+
|  BAC| 1.17884953591E11|
| AAPL|  6.8046305767E10|
|   GE|  5.3023644686E10|
|    F|  4.3388129992E10|
|   FB|  4.3258314899E10|
| MSFT|  4.2641653597E10|
|  AMD|  4.0941468586E10|
|   MU|  3.8082749687E10|
| INTC|  3.6922332801E10|
| CSCO|  3.6075826186E10|
|  PFE|  3.5527608853E10|
|    T|  3.1850598073E10|
|  CHK|  3.1421758366E10|
|CMCSA|  3.0630585625E10|
|  FCX|  2.9648463667E10|
|    C|  2.6044628798E10|
|  WFC|  2.3305954532E10|
|  JPM|  2.0885592857E10|
|   RF|    2.06456505E10|
| NFLX|  2.0405882026E10|
+-----+-----------------+
only showing top 20 rows



Quantas ações começam com a letra “A”?

In [339]:
df5 = df_mod.groupBy('Name').count()
df5.show(), df5.count()

+----+-----+
|Name|count|
+----+-----+
|ALXN| 1259|
| AIV| 1259|
| AVY| 1259|
|BF.B| 1259|
| AXP| 1259|
|  CI| 1259|
| BLK| 1259|
| DGX| 1259|
|AAPL| 1259|
| ALL| 1259|
| CLX| 1259|
|CSCO| 1259|
| BMY| 1257|
| CMG| 1259|
| ADI| 1259|
| AGN| 1259|
|CHRW| 1259|
|AMGN| 1259|
| CCL| 1259|
|ANTM| 1259|
+----+-----+
only showing top 20 rows



(None, 505)

In [342]:
df5.filter(df5.Name.startswith('A')==True).count()

59

Com qual frequência o preço mais alto do dia da ação também é o preço de fechamento?

In [396]:
df_agrupado_dia = df_mod.select('date')
df_agrupado_dia.show()

+----------+
|      date|
+----------+
|2013-02-08|
|2013-02-11|
|2013-02-12|
|2013-02-13|
|2013-02-14|
|2013-02-15|
|2013-02-19|
|2013-02-20|
|2013-02-21|
|2013-02-22|
|2013-02-25|
|2013-02-26|
|2013-02-27|
|2013-02-28|
|2013-03-01|
|2013-03-04|
|2013-03-05|
|2013-03-06|
|2013-03-07|
|2013-03-08|
+----------+
only showing top 20 rows



In [389]:
df_agrupado_dia.columns

['date', 'close', 'max(high)']

In [390]:
df_agrupado_dia = df_agrupado_dia.withColumnRenamed('max(high)','maiorValorDia')

In [391]:
df_agrupado_dia.columns

['date', 'close', 'maiorValorDia']

In [392]:
df_agrupado_dia.filter(df_agrupado_dia.close==df_agrupado_dia.maiorValorDia).count()

609992

In [395]:
(1-(609992/619040))*100

1.4616179891444814

In [361]:
df_agrupado_dia = df_mod.groupBy('date').agg(count('*')).show()

+----------+--------+
|      date|count(1)|
+----------+--------+
|2013-03-14|     476|
|2014-05-27|     485|
|2015-05-01|     489|
|2016-08-17|     498|
|2017-12-05|     505|
|2014-07-14|     486|
|2014-12-11|     488|
|2015-02-09|     489|
|2015-02-27|     489|
|2016-04-22|     497|
|2016-08-08|     498|
|2017-02-24|     500|
|2017-05-11|     501|
|2017-10-20|     504|
|2017-12-22|     505|
|2013-08-20|     482|
|2015-02-26|     489|
|2015-11-20|     495|
|2016-07-06|     498|
|2013-11-08|     482|
+----------+--------+
only showing top 20 rows



In [348]:
df_agrupado_dia.filter(df_agrupado_dia.close==df_agrupado_dia)

+----------+---------+
|      date|max(high)|
+----------+---------+
|2013-03-14|    717.8|
|2014-05-27|  1259.84|
|2015-05-01|  1260.89|
|2016-08-17|  1429.27|
|2017-12-05|  1741.07|
|2014-07-14|   1236.0|
|2014-12-11|  1127.33|
|2015-02-09|  1039.36|
|2015-02-27|  1246.39|
|2016-04-22|  1349.57|
|2016-08-08|  1418.58|
|2017-02-24|  1640.68|
|2017-05-11|  1824.76|
|2017-10-20|  1946.89|
|2017-12-22|  1786.22|
|2013-08-20|   942.15|
|2015-02-26|   1264.0|
|2015-11-20|   1296.0|
|2016-07-06|  1294.43|
|2013-11-08|  1077.14|
+----------+---------+
only showing top 20 rows



In [181]:
df_mod.select(max(df_mod.high)).show()

+---------+
|max(high)|
+---------+
|   999.75|
+---------+



In [397]:
df_aple = df_mod.filter(df_mod.Name =='AAPL')
df_aple.show()

+----------+-------+-------+-------+-------+------------+----+
|      date|   open|   high|    low|  close|      volume|Name|
+----------+-------+-------+-------+-------+------------+----+
|2013-02-08|68.4014|68.4014|68.4014|68.4014|1.58168416E8|AAPL|
|2013-02-11|69.2771|69.2771|69.2771|69.2771|1.29029425E8|AAPL|
|2013-02-12|68.9114|68.9114|68.9114|68.9114|1.51829363E8|AAPL|
|2013-02-13|67.6628|67.6628|67.6628|67.6628|1.18721995E8|AAPL|
|2013-02-14|67.3771|67.3771|67.3771|67.3771| 8.8809154E7|AAPL|
|2013-02-15|67.1656|67.1656|67.1656|67.1656| 9.7924631E7|AAPL|
|2013-02-19|66.1042|66.1042|66.1042|66.1042|1.08854046E8|AAPL|
|2013-02-20|65.3842|65.3842|65.3842|65.3842|1.18891367E8|AAPL|
|2013-02-21|64.1671|64.1671|64.1671|64.1671|1.11596821E8|AAPL|
|2013-02-22|64.5142|64.5142|64.5142|64.5142| 8.2583823E7|AAPL|
|2013-02-25|65.0171|65.0171|65.0171|65.0171| 9.2899597E7|AAPL|
|2013-02-26|64.5056|64.5056|64.5056|64.5056|1.25096657E8|AAPL|
|2013-02-27|64.6342|64.6342|64.6342|64.6342|1.46674682E

In [191]:
df_aple.filter(df_aple.date == '2015-08-24').show()

+----------+-----+-----+----+------+------------+----+
|      date| open| high| low| close|      volume|Name|
+----------+-----+-----+----+------+------------+----+
|2015-08-24|94.87|108.8|92.0|103.12|1.62206292E8|AAPL|
+----------+-----+-----+----+------+------------+----+



In [192]:
df_aple.filter(df_aple.date == '2018-02-06').show()

+----------+------+------+-----+------+-----------+----+
|      date|  open|  high|  low| close|     volume|Name|
+----------+------+------+-----+------+-----------+----+
|2018-02-06|154.83|163.72|154.0|163.03|6.8243838E7|AAPL|
+----------+------+------+-----+------+-----------+----+



df_aple.filter(df_aple.date == '2017-05-08').show()

In [194]:
df_aple.filter(df_aple.date == '2014-12-09').show()

+----------+------+-----+------+------+-----------+----+
|      date|  open| high|   low| close|     volume|Name|
+----------+------+-----+------+------+-----------+----+
|2014-12-09|110.19|114.3|109.35|114.12|6.0208036E7|AAPL|
+----------+------+-----+------+------+-----------+----+



Em média, qual o volume diário de transações das ações da AAPL?

In [400]:
df_aple.groupBy('Name').agg(mean('volume')).show()

+----+-------------------+
|Name|        avg(volume)|
+----+-------------------+
|AAPL|5.404789973550437E7|
+----+-------------------+



Qual a ação menos negociada da bolsa, em volume de transações?

In [405]:
df_mod.groupBy('Name').agg(sum('volume').alias('Volume_totalizado')).sort(desc('Volume_totalizado')).tail(5)

[Row(Name='WLTW', Volume_totalizado=414734590.0),
 Row(Name='AZO', Volume_totalizado=397585127.0),
 Row(Name='MTD', Volume_totalizado=214489754.0),
 Row(Name='BHF', Volume_totalizado=144878392.0),
 Row(Name='APTV', Volume_totalizado=92947779.0)]